In [27]:
!python --version

Python 3.9.6


In [28]:
from typing import List, Optional, Tuple
from random import uniform, randrange

```
class Gene {
  public Float value;
  public Gene(Float value) { this.value = value; }
  public static Gene get_random(Pair<Float, Float> values_range) {
    random_value = RandomFloat(values_range.getValue0(), values_range.getValue1();
    return new Gene(random_value);}}
```

In [29]:
class Gene:  # == 'trait'/'feature'
    def __init__(self, value: float):
        self.value: float = value
    @classmethod
    def get_random(cls, values_range: Tuple[float, float]):
        random_value = uniform(values_range[0], values_range[1])
        return cls(random_value)

In [30]:
# class Genome {
#   public ArrayList<Gene> genes;
#   public Genome(ArrayList<Gene> genes) { this.genes = genes; }}

class Genome:  # == 'set of traits/features'
    def __init__(self, genes: List[Gene]):
        self.genes: List[Gene] = genes
    @classmethod
    def get_random(cls, genome_lenght: int, values_range: Tuple[float, float]):
        genes: List[Gene] = []
        for gene_index in range(genome_lenght):
            genes.append(Gene.get_random(values_range))
        return cls(genes)
    def crossover(self, other_genome: Genome) -> Genome:
        result_genes: List[Gene] = []
        for gene_index in range(len(self.genes)):
            if uniform(0.0, 1.0) < 0.5:
                result_genes.append(self.genes[gene_index])
            else:
                result_genes.append(other_genome[gene_index])
        return Genome(result_genes)
    def mutate(self, probability: float, values_range: Tuple[float, float]):
        if uniform(0.0, 1.0) < probability:
            mutating_gene_index = randrange(len(self.genes))
            self.genes[mutating_gene_index] = uniform(values_range[0], values_range[1])

In [31]:
# class Individual {
#   public Genome genome;
#   public Float score;
#   public Individual(Genome genome) { this.genome = genome }}

class Individual:  # 'genome representative'
    def __init__(self, genome: Genome):
        self.genome: Genome = genome
        self.score: Optional[float] = None
    @classmethod
    def get_random(cls, genome_lenght: int, values_range: Tuple[float, float]):
        genome = Genome.get_random(genome_lenght, values_range)
        return cls(genome)

In [47]:
# class Population {
#   public ArrayList<Individual> individuals;
#   public Population(ArrayList<Individual> individuals) { this.individuals = individuals }}

class Population:  # 'set of Individuals'
    def __init__(self, individuals: List[Individual]):
        self.individuals: List[Individual] = individuals
    def get_the_best(self) -> Individual:
        return self.individuals.sort(key=lambda individual:individual.score)[0]
    def get_top_scored(self, percentage: float) -> List[Individual]:
        n_best: int = (len(self.individuals) * percentage).ceil()
        return self.individuals[:n_best]

In [56]:
def generate_init_population(population_size: int, genome_lenght: int, values_range: Tuple[float, float]):
    generated_individuals: List[Individual] = []
    for individual_index in range(population_size):
        generated_individuals.append(Individual.get_random(genome_lenght, values_range))
    return Population(generated_individuals)

In [57]:
def generate_new_individuals(individuals: List[Individual]) -> List[Individual]:
    pass  # TODO

In [55]:
class ProblemAbstractClass:
    def __init__(self, population_size: int, genome_lenght: int, values_range: Tuple[float, float], crossover_percentage: float, survivors_percentage: float, mutation_probability: float, target_score: float, iterations_limit: int):
        self.population_size: int = population_size
        self.genome_lenght: int = genome_lenght
        self.values_range: Tuple[float, float] = values_range
        self.crossover_percentage: float = crossover_percentage
        self.survivors_percentage: float = survived_percentage
        self.mutation_probability: float = mutation_probability
        self.target_score: float = target_score
        self.iterations_limit: int = iterations_limit
        # generate initial population
        self.current_population: Population = generate_init_population(self.population_size, self.genome_lenght, self.values_range)
            
    # THIS IS THE GENETIC ALGORITHM
    def solve(self) -> Individual:
        # rate individuals
        self.rate_individuals()
        iteration: int = 0
        # if stop condition is met - return the best individual from current_population
        while current_population.get_the_best().score < self.target_score and iteration < self.iterations_limit:
            # choose individuals for crossover
            crossover_individuals: List[Individual] = current_population.get_top_scored(self.crossover_percentage)
            # generate children subpopulation by crossover and mutation
            children: List[Individual] = generate_new_individuals(crossover_individuals)
            # create new population as children subpopulation plus the best individuals from current_population
            survivors: List[Individual] = current_population.get_top_scored(self.survivors_percentage)
            self.current_population = Population(children + survivors)
            # rate individuals
            self.rate_individuals()
        return current_population.get_best()
    
    def rate_individuals(self):
        for individual in self.current_population.individuals:
            if individual.score is None:  # if it's not a survivor (avoid rating multiple times)
                self.rate_one_individual(individual)
    def rate_one_individual(self, individual): ...

IndentationError: expected an indented block (2262216412.py, line 2)